In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
import pandas as pd
import pickle
import serial

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose 

# 3. Train Custom Model Using Scikit Learn

In [2]:
df = pd.read_csv('coords_all.csv')

In [3]:
df[df['spine']==1] #왼쪽 치우침

,spine,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,x13,y13,z13,v13,StnX,StnY,X0,Y0,X,Y
5,1,0.636111,0.548702,-1.273232,0.998880,0.667076,0.491745,-1.178279,0.997638,0.684673,...,0.315934,0.855035,-0.539416,0.988677,156,250,28,50,116,-328
6,1,0.691333,0.605470,-1.223022,0.998321,0.719905,0.548739,-1.125882,0.996888,0.734329,...,0.289917,0.835516,-0.615826,0.993998,156,252,29,50,117,-329
7,1,0.703984,0.614274,-1.225254,0.997541,0.730344,0.558983,-1.129225,0.995548,0.743108,...,0.297610,0.838353,-0.636646,0.993398,150,250,28,50,122,-322
8,1,0.701412,0.607026,-1.198113,0.997642,0.728034,0.552146,-1.100823,0.995681,0.742170,...,0.299361,0.838692,-0.607690,0.993594,147,255,29,50,129,-323
9,1,0.687203,0.574255,-1.203558,0.998359,0.717374,0.517160,-1.110130,0.996831,0.733600,...,0.310026,0.835661,-0.520373,0.994771,152,256,29,51,126,-328
10,1,0.655819,0.559890,-1.251236,0.998822,0.692937,0.503027,-1.158664,0.997620,0.712574,...,0.316416,0.835803,-0.490797,0.994649,155,256,29,52,124,-330
23,1,0.727915,0.568268,-1.053504,0.999163,0.758766,0.511695,-0.957304,0.998573,0.775621,...,0.341177,0.828931,-0.466214,0.992880,155,239,30,54,108,-310
24,1,0.732434,0.567754,-1.059323,0.999266,0.764062,0.512149,-0.961780,0.998691,0.781135,...,0.342073,0.836350,-0.475848,0.995241,166,238,31,54,95,-319
25,1,0.730400,0.567951,-1.100515,0.999358,0.763390,0.512380,-1.005844,0.998818,0.780646,...,0.342704,0.835507,-0.488405,0.996111,168,240,31,54,95,-323
26,1,0.729657,0.571699,-1.070943,0.999288,0.763117,0.515229,-0.977332,0.998682,0.780513,...,0.344476,0.834457,-0.477074,0.995944,169,240,31,54,94,-324


In [4]:
df[df['spine']==2] #오른쪽 치우침

,spine,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,x13,y13,z13,v13,StnX,StnY,X0,Y0,X,Y
15,2,0.550053,0.538226,-1.403773,0.993610,0.574969,0.467294,-1.342669,0.991884,0.598220,...,0.359726,0.901203,-0.437524,0.955426,174,244,29,51,92,-338
16,2,0.523286,0.553901,-1.375170,0.995554,0.548858,0.476140,-1.321058,0.993097,0.575717,...,0.363487,0.912521,-0.389488,0.972986,165,208,26,46,63,-301
17,2,0.511620,0.557521,-1.253204,0.996481,0.535519,0.479884,-1.200223,0.993911,0.561876,...,0.357117,0.917542,-0.294485,0.979787,166,205,29,48,58,-294
18,2,0.493464,0.561100,-1.139336,0.999317,0.515612,0.484068,-1.091580,0.998685,0.540343,...,0.345953,0.927004,-0.253680,0.995095,171,215,29,49,64,-308
19,2,0.500114,0.551610,-1.166438,0.999771,0.524848,0.475644,-1.114725,0.999558,0.550796,...,0.342291,0.921374,-0.310745,0.997702,178,221,30,50,63,-319
34,2,0.492449,0.536425,-1.151968,0.999502,0.514584,0.461872,-1.092452,0.999146,0.539818,...,0.331919,0.924978,-0.276306,0.995902,169,240,31,59,99,-319
35,2,0.493011,0.535158,-1.125793,0.999689,0.514784,0.460334,-1.070806,0.999451,0.539834,...,0.334285,0.920205,-0.270448,0.996856,170,223,30,52,75,-311
36,2,0.516563,0.516166,-1.237794,0.999723,0.540278,0.449623,-1.164451,0.999437,0.562901,...,0.328537,0.909348,-0.349991,0.997426,172,226,30,54,78,-314
37,2,0.492979,0.561585,-1.100869,0.999173,0.515113,0.484016,-1.055894,0.998409,0.539652,...,0.346100,0.925916,-0.239882,0.994325,174,231,31,55,81,-319
38,2,0.493756,0.560257,-1.113112,0.999484,0.515982,0.483896,-1.068637,0.999025,0.540826,...,0.345175,0.926946,-0.255105,0.995654,175,233,31,56,83,-321


In [5]:
X = df.drop('spine',axis=1) #shol을 제외한 입력 feature만 남김.
y = df['spine']

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123) #어깨

In [7]:
y_test

72    0
53    0
59    0
52    0
51    0
63    0
68    1
23    1
8     1
4     0
58    2
9     1
71    0
42    0
65    1
44    1
1     0
24    1
31    0
54    2
56    2
14    0
Name: spine, dtype: int64

https://camo.githubusercontent.com/37bc1d0f546f2dd001798a0f66057c9c907e6b5efae95a068bcce8c852c2e234/68747470733a2f2f692e696d6775722e636f6d2f336a38425064632e706e67

In [8]:
df

,spine,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,x13,y13,z13,v13,StnX,StnY,X0,Y0,X,Y
0,0,0.565311,0.522980,-1.450256,0.996940,0.603264,0.457090,-1.376138,0.995170,0.623617,...,0.323954,0.878036,-0.528042,0.976571,161,247,30,51,107,-327
1,0,0.566478,0.522845,-1.421807,0.997090,0.604397,0.456763,-1.344025,0.995315,0.624477,...,0.319657,0.884477,-0.504303,0.978333,162,250,30,52,110,-330
2,0,0.565075,0.518007,-1.411711,0.997514,0.602877,0.454484,-1.330005,0.995809,0.623085,...,0.316250,0.886326,-0.504217,0.979388,160,246,30,52,108,-324
3,0,0.565575,0.518130,-1.403396,0.997495,0.603070,0.454441,-1.322353,0.995740,0.623184,...,0.315378,0.886709,-0.499001,0.979245,163,249,30,52,108,-330
4,0,0.569317,0.520454,-1.292739,0.998043,0.604864,0.456249,-1.208947,0.996691,0.624150,...,0.317728,0.881516,-0.418823,0.979229,166,245,30,52,101,-329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,1,0.638881,0.539872,-1.200169,0.999185,0.671454,0.484758,-1.106733,0.998538,0.689324,...,0.317797,0.856111,-0.460463,0.995045,181,249,21,49,96,-360
69,0,0.532206,0.516137,-1.325961,0.997986,0.567318,0.455442,-1.234663,0.998074,0.587338,...,0.301883,0.895617,-0.315608,0.928452,184,249,22,53,96,-358
70,0,0.533456,0.516018,-1.304827,0.998330,0.568439,0.455465,-1.215089,0.998535,0.588193,...,0.302890,0.893256,-0.307524,0.917230,185,248,22,53,94,-358
71,0,0.533559,0.517672,-1.330495,0.997986,0.568301,0.456553,-1.237293,0.998252,0.588130,...,0.301337,0.894897,-0.343794,0.898795,184,253,22,54,101,-361


In [9]:
df.mean()

spine      0.712329
x1         0.595383
y1         0.536952
z1        -1.277631
v1         0.998229
x2         0.626621
y2         0.472859
z2        -1.193688
v2         0.997403
x3         0.646471
y3         0.473673
z3        -1.194268
v3         0.997472
x4         0.664570
y4         0.475170
z4        -1.194508
v4         0.997087
x5         0.564641
y5         0.470368
z5        -1.205164
v5         0.997610
x6         0.542891
y6         0.469996
z6        -1.204914
v6         0.997901
x7         0.523148
y7         0.469680
z7        -1.205570
v7         0.998045
x8         0.687777
y8         0.509626
z8        -0.695341
v8         0.996589
x9         0.492974
y9         0.496581
z9        -0.733594
v9         0.998263
x10        0.632386
y10        0.615244
z10       -1.091390
v10        0.998422
x11        0.554113
y11        0.610148
z11       -1.103823
v11        0.998909
x12        0.826694
y12        0.886053
z12       -0.368490
v12        0.965682
x13        0.331149


In [10]:
for val in range(1,12+1):
    print(df['v{}'.format(val)].mean())

0.998228802260274
0.9974026721095889
0.997472033890411
0.9970871757534248
0.9976096871780822
0.9979009669315068
0.9980454657671232
0.9965886114246574
0.9982625906575342
0.9984215441369862
0.9989086111917808
0.9656819015479452


## 3.2 Train Machine Learning Classification Model

In [11]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(),LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [12]:
fit_models={}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [13]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [14]:
fit_models['rf'].predict(X_test)

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 0],
      dtype=int64)

In [15]:
y_test

72    0
53    0
59    0
52    0
51    0
63    0
68    1
23    1
8     1
4     0
58    2
9     1
71    0
42    0
65    1
44    1
1     0
24    1
31    0
54    2
56    2
14    0
Name: spine, dtype: int64

## 3.3 Evaluate and Serialize Model

In [16]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test.values,yhat),
    precision_score(y_test.values,yhat,average=None),
    recall_score(y_test.values,yhat,average=None))

lr 1.0 [1. 1. 1.] [1. 1. 1.]
rc 1.0 [1. 1. 1.] [1. 1. 1.]
rf 1.0 [1. 1. 1.] [1. 1. 1.]
gb 1.0 [1. 1. 1.] [1. 1. 1.]


In [17]:
yhat = fit_models['rf'].predict(X_test)

In [18]:
yhat[:10]

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0], dtype=int64)

In [19]:
y_test

72    0
53    0
59    0
52    0
51    0
63    0
68    1
23    1
8     1
4     0
58    2
9     1
71    0
42    0
65    1
44    1
1     0
24    1
31    0
54    2
56    2
14    0
Name: spine, dtype: int64

In [20]:
with open('Spine.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

# 4. Make Detections with Model

In [21]:
with open('Spine.pkl','rb') as f:
    model = pickle.load(f)

In [22]:
landmarks=['spine']
for val in range(1,13+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [23]:
landmarks

['spine',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13']

In [24]:
import serial

#port = "COM12"
port = "COM6"
brate = 9600 #boudrate
cmd = 't'

seri = serial.Serial(port, baudrate = brate, timeout = None)
print(seri.name)

seri.write(cmd.encode())

counter =0

#VISUALIZE DEGREE
cap =cv2.VideoCapture(0) #setting video capture device(number은 웹캠을 대표하는 숫자)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret, frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
        if not ret:
            print("Error: Unable to capture frame from the camera.")
        #Detect stuff and render

        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환
        
         #Extract landmarks
        row =[]
        try:  
            for k in range(12+1):
                row.append(results.pose_landmarks.landmark[k].x)
                row.append(results.pose_landmarks.landmark[k].y)
                row.append(results.pose_landmarks.landmark[k].z)
                row.append(results.pose_landmarks.landmark[k].visibility)

            
            row = np.array(row).flatten()
            
            X = pd.DataFrame([row],columns=landmarks[1:])


            if seri.in_waiting != 0 :
                content1 = seri.readline().decode('ascii')
                X['StnX'] = float(content1[:-2])
                content2 = seri.readline().decode('ascii')
                X['StnY'] = float(content2[:-2])
                content3 = seri.readline().decode('ascii')
                X['X0'] = float(content3[:-2])
                content4 = seri.readline().decode('ascii')
                X['Y0'] = float(content4[:-2])
                content5 = seri.readline().decode('ascii')
                X['X'] = float(content5[:-2])
                content6 = seri.readline().decode('ascii')
                X['Y'] = float(content6[:-2])

            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]

            if body_language_class == 0 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'Good'
            elif current_stage == 'Good' and body_language_class ==1 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'left'
                counter +=1
            elif current_stage == 'Good' and body_language_class ==2 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'right'
                counter +=1


            #Setup status bow
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1) 


            #지금 상태
            cv2.putText(image,'CLASS',(15,12),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image, 'left' if body_language_class == 1 else 'right' if body_language_class == 2 else 'Good',(15,40)
                        ,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
            #Stage data 
            cv2.putText(image,'Count',(180,12),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)

            cv2.putText(image,str(counter),(175,40),
                        cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            # Render detections
            #drawing 툴을 사용해서 감지된 자세포인트와 이들 간의 포인트를 연결해 보여준다.
            #mp_drawing.DrawingSpec은 관절부위와 bone 부분의 색깔, 굵기 등을 지정해준다.
            mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(155,117,166),thickness=2,circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(195,116,230),thickness=2,circle_radius=2))


        except: #error가 있으면 실행x
            pass
        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

COM6


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
